In [1]:
## Load required libraries
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import scipy as sp
import scipy.optimize as sp_opt
import pandas as pd
import time

#tf.debugging.set_log_device_placement(True);

In [18]:
## Load data
data = pd.read_excel("~/Desktop/enigh_income_data_labor.xls", header = None)
print("\nFirst 10 rows of data:")
print(data.head(10))
data = data.to_numpy()
# Transform to monthly income in thousands
data = data / np.array([3000,1])

# Cast into Tensorflow objects
Y = tf.constant(data[:,0], dtype = tf.float32)
Weights = tf.constant(data[:,1], dtype = tf.float32)
expanded_n = tf.math.reduce_sum(Weights)

# Print Tensorflow objects
print("\nFirst 10 elements of Y tensor:")
tf.print(Y[0:9])
print("\nFirst 10 elements of Weights tensor:")
tf.print(Weights[0:9])
print("\nExpanded number of observations in data:")
tf.print(expanded_n)
print("\nAverage income in data:")
tf.print(tf.reduce_sum(Weights*Y)/expanded_n)


First 10 rows of data:
              0     1
0   5869.560059  1537
1  25679.339844  1537
2  21277.160156  1537
3   7092.379883  1537
4  54293.468750  1537
5  30091.289062  1537
6  32624.990234  1537
7  27880.429688  1537
8  26657.599609  1537
9   5282.600098  1215

First 10 elements of Y tensor:
[1.95652 8.55978 7.09238672 ... 10.8749971 9.2934761 8.88586617]

First 10 elements of Weights tensor:
[1537 1537 1537 ... 1537 1537 1537]

Expanded number of observations in data:
38294416

Average income in data:
5.43961477


In [37]:
## Test likelihood function of the Generalized Gamma distribution in Tensorflow
test_param = tf.constant([4., 1.0, 1.0], dtype = tf.float32)

# Define target negative log-likelihood function without constraints
@tf.function
def target_GeneralizedGamma(param_vec, Y = Y, Weights = Weights, n = expanded_n):
    # Retrieve distribution parameters
    a, d, p = tf.split(param_vec, 3, axis = 0)
    
    # Normalize weights
    #weights = weights / n
    #tf.print(weights[0:9])
    
    # Compute negative log-likelihood
    ll = -(n*tf.math.log((p/a**d)/tf.math.exp(tf.math.lgamma(d/p))) + (d-1)*tf.math.reduce_sum(Weights*tf.math.log(Y)) - tf.math.reduce_sum(Weights*(Y/a)**p))
    
    return tf.squeeze(ll)

# Return target function and its gradient
@tf.function
def target_grad_GeneralizedGamma(param_vec):
    return tfp.math.value_and_gradient(target_GeneralizedGamma, param_vec)

print( target_grad_GeneralizedGamma(test_param) )

(<tf.Tensor: shape=(), dtype=float32, numpy=105164056.0>, <tf.Tensor: shape=(3,), dtype=float32, numpy=array([-3445576., -9397222., 25728128.], dtype=float32)>)


In [38]:
## Minimize negative log-likelihood using BFGS in Tensorflow
start_param = test_param

optim_results = tfp.optimizer.bfgs_minimize(target_grad_GeneralizedGamma, start_param, tolerance = 1e-10)
print("\nOutput of optimizer:")
tf.print(optim_results)
est_params = optim_results.position.numpy()

a_fitted = tf.constant(est_params[0], dtype = tf.float64)
d_fitted = tf.constant(est_params[1], dtype = tf.float64)
p_fitted = tf.constant(est_params[2], dtype = tf.float64)

print("\nEstimated parameters:")
print(est_params)

# Compute fitted mean
mu_fitted = a_fitted * tf.math.exp(tf.math.lgamma((d_fitted+1)/p_fitted)) / tf.math.exp(tf.math.lgamma(d_fitted/p_fitted))
print("\nMean of fitted distribution:")
tf.print(mu_fitted)


Output of optimizer:
BfgsOptimizerResults(converged=1, failed=0, num_iterations=11, num_objective_evaluations=47, position=[3.85739279 1.02209401 0.82695514], objective_value=102917600, objective_gradient=[2 25 -28], inverse_hessian_estimate=[[0.000245528645 -1.82728709e-05 1.86315447e-05]
 [-1.82728709e-05 1.37505299e-06 -1.37900315e-06]
 [1.86315447e-05 -1.37900315e-06 1.42219676e-06]])

Estimated parameters:
[3.8573928  1.022094   0.82695514]

Mean of fitted distribution:
5.42960381096186


In [40]:
## Minimize negative log-likelihood using gradient descent
@tf.function
def SGDfunction(a, d, p, lambda1, Y = Y, Weights = Weights, n = expanded_n):
    target = -(n*tf.math.log((p/a**d)/tf.math.exp(tf.math.lgamma(d/p))) + (d-1)*tf.math.reduce_sum(Weights*tf.math.log(Y)) - tf.math.reduce_sum(Weights*(Y/a)**p)) \
        + lambda1 * ((a * tf.math.exp(tf.math.lgamma((d+1)/p)) / tf.math.exp(tf.math.lgamma(d/p))) - 1)
    
    return tf.squeeze(target)

@tf.function
def SGDfunction_minimize(Y = Y, Weights = Weights, n = expanded_n):
    target = -(n*tf.math.log((p/a**d)/tf.math.exp(tf.math.lgamma(d/p))) + (d-1)*tf.math.reduce_sum(Weights*tf.math.log(Y)) - tf.math.reduce_sum(Weights*(Y/a)**p)) \
        + lambda1 * ((a * tf.math.exp(tf.math.lgamma((d+1)/p)) / tf.math.exp(tf.math.lgamma(d/p))) - 1)
    
    return tf.squeeze(target)

# Generate tensors with initial values
def SGD_initial_values():
    a = tf.Variable(4.00) 
    d = tf.Variable(1.00)
    p = tf.Variable(1.00) 
    lambda1 = tf.Variable(1e5*1.0) 
    return a, d, p, lambda1

a, d, p, lambda1 = SGD_initial_values()

opt = tf.keras.optimizers.SGD(learning_rate = 3e-9)
tic = time.time()
for i in range(10001):
    if i%10000 == 0:
        print("\nGradient at iteration " + str(i))
        with tf.GradientTape() as tape:
            function = SGDfunction(a, d, p, lambda1)
        tf.print(tape.gradient(function, [a, d, p, lambda1]))
        print("Parameter values at iteration " + str(i))
        tf.print(a)
        tf.print(d)
        tf.print(p)
        tf.print(lambda1)
    opt.minimize(SGDfunction_minimize, var_list = [a, d, p, lambda1])
print("\nGradient Descent took "+str(time.time()-tic)+" seconds")

# Compute fitted mean
mu_fitted_2 = a * tf.math.exp(tf.math.lgamma((d+1)/p)) / tf.math.exp(tf.math.lgamma(d/p))
print("\nMean of fitted distribution:")
tf.print(mu_fitted_2)


Gradient at iteration 0
[-3345576, -8997222, 25159012, 3]
Parameter values at iteration 0
4
1
1
100000

Gradient at iteration 10000
[2189, -161.421875, 192, 4.34211063]
Parameter values at iteration 10000
3.83816457
1.02017117
0.830833733
100000

Gradient Descent took 20.938907861709595 seconds

Mean of fitted distribution:
5.34211159


In [41]:
## Perform Maximum Likelihood Estimation with constraints using BFGS in Tensorflow
start_param_constrained = tf.constant([1.5, .5], dtype = tf.float32)

constrained_mean = 8.
top_threshold = 200.
constrained_density = 0.0005

# Define target negative log-likelihood function with constraints
@tf.function
def target_GeneralizedGamma_Constrained(param_vec_constrained, Y = Y, Weights = Weights, n = expanded_n):
    # Retrieve distribution parameters
    d, p = tf.split(param_vec_constrained, 2, axis = 0)
    
    # Constraint over the mean of the distribution
    a = constrained_mean / (tf.math.exp(tf.math.lgamma((d+1)/p)) / tf.math.exp(tf.math.lgamma(d/p)))
    
    # Compute negative log-likelihood
    ll = -(n*tf.math.log((p/a**d)/tf.math.exp(tf.math.lgamma(d/p))) + (d-1)*tf.math.reduce_sum(Weights*tf.math.log(Y)) - tf.math.reduce_sum(Weights*(Y/a)**p))
    
    return tf.squeeze(ll)

# Return target function and its gradient
@tf.function
def target_grad_GeneralizedGamma_Constrained(param_vec_constrained):
    return tfp.math.value_and_gradient(target_GeneralizedGamma_Constrained, param_vec_constrained)

# Minimize negative log-likelihood via BFGS
optim_results_constrained = tfp.optimizer.bfgs_minimize(target_grad_GeneralizedGamma_Constrained, start_param_constrained, tolerance = 1e-10)
print("\nOutput of optimizer:")
tf.print(optim_results_constrained)
est_params_constrained = optim_results_constrained.position.numpy()

d_fitted_constrained = tf.constant(est_params_constrained[0], dtype = tf.float32)
p_fitted_constrained = tf.constant(est_params_constrained[1], dtype = tf.float32)
a_fitted_constrained = constrained_mean / (tf.math.exp(tf.math.lgamma((d_fitted_constrained+1)/p_fitted_constrained)) / tf.math.exp(tf.math.lgamma(d_fitted_constrained/p_fitted_constrained)))

print("\nEstimated parameters:")
tf.print(a_fitted_constrained)
tf.print(d_fitted_constrained)
tf.print(p_fitted_constrained)

print("\nConstraint is satisfied. Mean equals:")
mu_fitted_constrained = a_fitted_constrained * tf.math.exp(tf.math.lgamma((d_fitted_constrained+1)/p_fitted_constrained)) / tf.math.exp(tf.math.lgamma(d_fitted_constrained/p_fitted_constrained))
tf.print(mu_fitted_constrained)

# Verify second constraint
@tf.function
def CDF_GeneralizedGamma(x, a, p, d):
    # Using the inverse gamma distribution this way yields the same as the CDF for the generalized gamma distribution
    return tf.math.igammac( (d/p), ((x/a)**p) )

# Density above threshold:
print("\nDensity above threshold of " + str(top_threshold) +":")
tf.print( CDF_GeneralizedGamma(top_threshold, a_fitted_constrained, p_fitted_constrained, d_fitted_constrained) )


Output of optimizer:
BfgsOptimizerResults(converged=1, failed=0, num_iterations=7, num_objective_evaluations=45, position=[1.09829903 0.588815212], objective_value=104750208, objective_gradient=[-478 -708], inverse_hessian_estimate=[[1.95692238e-07 -1.28930125e-07]
 [-1.28930125e-07 1.14102107e-07]])

Estimated parameters:
2.13125801
1.09829903
0.588815212

Constraint is satisfied. Mean equals:
8

Density above threshold of 200.0:
5.68335781e-06


In [42]:
## Perform Maximum Likelihood Estimation with constraints using BFGS in Scipy

start_param_scipy = test_param.numpy()

mydata = [data[:,0], data[:,1], np.sum(data[:,1])]

def objective_GeneralizedGamma(params, mydata):
    # Retrieve parameters
    a = params[0]
    d = params[1]
    p = params[2]
    
    # Retrieve observations
    Y = mydata[0]
    Weights = mydata[1]
    n = mydata[2]
    
    # Compute negative log-likelihood
    ll = -(n*np.log((p/a**d)/sp.special.gamma(d/p)) + (d-1)*np.sum(Weights*np.log(Y)) - np.sum(Weights*(Y/a)**p))
    
    return ll

def Mean_GeneralizedGamma(params):
    a = params[0]
    d = params[1]
    p = params[2]
    
    mean = a * sp.special.gamma((d+1)/p) / sp.special.gamma(d/p)
    return mean

def constrained_mean_fun(params):
    cons_mean = Mean_GeneralizedGamma(params) - constrained_mean
    return cons_mean

def Right_CDF_GeneralizedGamma(params):
    a = params[0]
    d = params[1]
    p = params[2]
    
    right_cdf = sp.special.gammaincc(d/p, (top_threshold/a)**p)
    return right_cdf

def constrained_CDF_fun(params):
    cons_CDF = Right_CDF_GeneralizedGamma(params) - constrained_density
    return cons_CDF

# Firstly, test unconstrained optimization
#print( objective_GeneralizedGamma(start_param_scipy, mydata) )
myoptions = {'disp' : True, 'maxiter' : 1000}
unc_results = sp_opt.minimize(objective_GeneralizedGamma, start_param_scipy, args = mydata, options = myoptions)
print("\nUnconstrained solution:")
print(unc_results.x)
print("\nUnconstrained mean of the fitted distribution:")
unc_fitted_mean = Mean_GeneralizedGamma(unc_results.x)
print(unc_fitted_mean)
print("\n")

# Then, perform optimization over the constrained mean
con_start_param_scipy = [a_fitted_constrained, d_fitted_constrained, p_fitted_constrained]
myconstraints = ({'type': 'eq', 'fun' : constrained_mean_fun})
con_results = sp_opt.minimize(objective_GeneralizedGamma, con_start_param_scipy, args = mydata, constraints = myconstraints, options = myoptions)
print("\nConstrained solution:")
print(con_results.x)
print("\nConstrained mean of the fitted distribution:")
con_fitted_mean = Mean_GeneralizedGamma(con_results.x)
print(con_fitted_mean)
print("\nDensity above threshold of " + str(top_threshold) +":")
print(Right_CDF_GeneralizedGamma(con_results.x))
print("\n")

# Finally, perform optimization over the constrained mean and the constrained cdf
con_start_param_scipy = [a_fitted_constrained, d_fitted_constrained, p_fitted_constrained]
myconstraints = ({'type': 'eq', 'fun' : constrained_mean_fun}, {'type': 'eq', 'fun' : constrained_CDF_fun})
con_results = sp_opt.minimize(objective_GeneralizedGamma, con_start_param_scipy, args = mydata, constraints = myconstraints, options = myoptions)
print("\nConstrained solution:")
print(con_results.x)
print("\nConstrained mean of the fitted distribution:")
con_fitted_mean = Mean_GeneralizedGamma(con_results.x)
print(con_fitted_mean)
print("\nDensity above threshold of " + str(top_threshold) +":")
print(Right_CDF_GeneralizedGamma(con_results.x))
print("\n")


         Current function value: 102917601.255001
         Iterations: 18
         Function evaluations: 126
         Gradient evaluations: 25

Unconstrained solution:
[3.85740727 1.02209275 0.82695621]

Unconstrained mean of the fitted distribution:
5.429602872289771


Optimization terminated successfully.    (Exit mode 0)
            Current function value: 104750204.50078872
            Iterations: 4
            Function evaluations: 45
            Gradient evaluations: 4

Constrained solution:
[2.13132661 1.09831352 0.58882419]

Constrained mean of the fitted distribution:
7.999999698136261

Density above threshold of 200.0:
5.681601776561191e-06




C:\Users\ezequiel\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: RuntimeWarning: overflow encountered in double_scalars
C:\Users\ezequiel\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in log
C:\Users\ezequiel\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\ezequiel\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in power
C:\Users\ezequiel\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in double_scalars


Optimization terminated successfully.    (Exit mode 0)
            Current function value: 105688839.12171128
            Iterations: 252
            Function evaluations: 1630
            Gradient evaluations: 252

Constrained solution:
[2.16172617e-04 2.10199194e+00 2.27016312e-01]

Constrained mean of the fitted distribution:
7.999999960693105

Density above threshold of 200.0:
0.0004999999794301703


